In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import itertools
import random

# Define the number of bits
no_bit = 3

# Generate all possible binary strings of length no_bit
ds = [''.join(p) for p in itertools.product('01', repeat=no_bit)][1:]

# Generate random labels for each binary string
labels = [random.randint(0, 1) for i in range(len(ds))]

# Create the dataset
data = [[int(char) for char in string] + [label] for string, label in zip(ds, labels)]

# Convert the dataset to a DataFrame
columns = [f"X{i+1}" for i in range(no_bit)] + ["Output"]
data_df = pd.DataFrame(data, columns=columns)

# Split the dataset into training and testing
X = data_df.iloc[:, :-1].values
y = data_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Define the perceptron function
def perceptron(X, y, learning_rate=0.1, max_epochs=100):
    weights = np.zeros(X.shape[1] + 1)
    for _ in range(max_epochs):
        for xi, target in zip(X, y):
            activation = np.dot(xi, weights[1:]) + weights[0]
            prediction = 1 if activation >= 0 else 0
            update = learning_rate * (target - prediction)
            weights[1:] += update * xi
            weights[0] += update
    return weights

# Train the perceptron model
weights = perceptron(X_train, y_train, learning_rate=0.1, max_epochs=100)

# Make predictions on the test set
y_pred = np.where(np.dot(X_test, weights[1:]) + weights[0] >= 0, 1, 0)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy * 100)


Accuracy: 66.66666666666666


In [ ]:
import numpy as np
import pandas as pd
from math import sqrt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
import random
from operator import ixor
from functools import reduce

no_bit = 3

# Generate all possible binary strings of length no_bit
ds = [''.join(p) for p in itertools.product('01', repeat=no_bit)][1:]

#print(ds)

# Generate random labels for each binary string
labels = [random.randint(0, 1) for i in range(len(ds))]
# Create the dataset
data = [[int(char) for char in string] + [label] for string, label in zip(ds, labels)]

#print(data)

# Convert the dataset to a DataFrame
columns = [f"X{i+1}" for i in range(no_bit)] + ["Output"]
data_df = pd.DataFrame(data, columns=columns)

#columns
#data_df.shape
#data_df.describe

df = data_df
data = data_df.values.tolist()
#data

weights = np.random.rand(len(data[0])-1)
#weights

threshold = 0.5
gain_term = 0.9

# Make a prediction with weights
def predict(row, weights):
    activation = -1*threshold
    for i in range(len(row)-1):
        activation += weights[i] * row[i]
    return 1.0 if activation >= 0.0 else 0.0
#for row in data:
    #print("desired ",row[-1],": actual ",predict(row,weights))

def adapt_weights(data,weights):
    start_time = time.time() # note starting time 
    i=0
    while(i<len(data)):
        #print("i",i)
        row = data[i]
        actual_output = predict(row,weights)

        desired_output = row[-1]

        #print(row,desired_output,actual_output,weights)

        if(desired_output==0 and actual_output==1):
            weights = [element1 - gain_term*element2 for (element1, element2) in zip(weights,row[:-1])]
            i=0
            continue


        elif(desired_output==1 and actual_output==0):
            weights = [element1 + gain_term*element2 for (element1, element2) in zip(weights,row[:-1])]
            i=0
            continue
        i=i+1
    
    end_time = time.time()
    print("training time, ",end_time-start_time," seconds")
    print()
        
    return weights


def predict_acc(train_percent,test_percent):
    
    train = df.sample(frac = train_percent/100,random_state = 42).values.tolist()
    
    test = []
    for row in data:
        if row not in train:
            test.append(row)
            
    
    
    
    weights = np.random.rand(len(train[0])-1)
    threshold = 0.5

    adapt_weights(train,weights)

    cnt = 0

    for row in test:

        actual_output = predict(row,weights)
        desired_output = row[-1]

        print("de",desired_output,"ac",actual_output)

        if(actual_output==desired_output):
            cnt+=1

    return cnt/len(test)*100.0

predict_acc(60,40)
predict_acc(70,30)
predict_acc(80,20)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import itertools
import random

# Define the number of bits
no_bit = 3

# Generate all possible binary strings of length no_bit
ds = [''.join(p) for p in itertools.product('01', repeat=no_bit)][1:]

# Generate random labels for each binary string
labels = [random.randint(0, 1) for i in range(len(ds))]

# Create the dataset
data = [[int(char) for char in string] + [label] for string, label in zip(ds, labels)]

# Convert the dataset to a DataFrame
columns = [f"X{i+1}" for i in range(no_bit)] + ["Output"]
data_df = pd.DataFrame(data, columns=columns)

# Split the dataset into training and testing
X = data_df.iloc[:, :-1].values
y = data_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Define the perceptron function
def perceptron(arr, w, th):
    flag = 0
    for i in range(arr.shape[0]):
        total_sum = 0.0
        for j in range(arr.shape[1]-1):
            total_sum += arr[i][j] * w[j]
        
        predicted_output = 0
        desired_output = arr[i][arr.shape[1]-1]
        
        if total_sum < th:
            predicted_output = 0
        else:
            predicted_output = 1
        
        if predicted_output == 0 and desired_output == 1:
            flag = 1
            for j in range(arr.shape[1]-1):
                w[j] += arr[i][j]
            break
        elif predicted_output == 1 and desired_output == 0:
            flag = 1
            for j in range(arr.shape[1]-1):
                w[j] -= arr[i][j]
            break
    
    if flag == 1:
        w = perceptron(arr, w, th)
    
    return w

# Train the perceptron model
weights = perceptron(X_train, np.random.rand(len(X_train[0])-1), 0.5)

# Make predictions on the test set
y_pred = np.where(np.dot(X_test, weights) >= 0, 1, 0)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy * 100)
print("Weights:", weights)


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import itertools
import random

# Define the number of bits
no_bit = 3

# Generate all possible binary strings of length no_bit
ds = [''.join(p) for p in itertools.product('01', repeat=no_bit)][1:]

# Generate random labels for each binary string
labels = [random.randint(0, 1) for i in range(len(ds))]

# Create the dataset
data = [[int(char) for char in string] + [label] for string, label in zip(ds, labels)]

# Convert the dataset to a DataFrame
columns = [f"X{i+1}" for i in range(no_bit)] + ["Output"]
data_df = pd.DataFrame(data, columns=columns)

# Split the dataset into training and testing
X = data_df.iloc[:, :-1].values
y = data_df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

# Define the perceptron function
def perceptron(arr, w, th):
    flag = 0
    for i in range(arr.shape[0]):
        total_sum = 0.0
        for j in range(arr.shape[1]-1):
            total_sum += arr[i][j] * w[j]
        
        predicted_output = 0
        desired_output = arr[i][arr.shape[1]-1]
        
        if total_sum < th:
            predicted_output = 0
        else:
            predicted_output = 1
        
        if predicted_output == 0 and desired_output == 1:
            flag = 1
            for j in range(arr.shape[1]-1):
                w[j] += arr[i][j]
            break
        elif predicted_output == 1 and desired_output == 0:
            flag = 1
            for j in range(arr.shape[1]-1):
                w[j] -= arr[i][j]
            break
    
    if flag == 1:
        w = perceptron(arr, w, th)
    
    return w

# Train the perceptron model
weights = perceptron(X_train, np.random.rand(len(X_train[0])-1), 0.5)

# Make predictions on the test set
y_pred = np.where(np.dot(X_test, weights) >= 0, 1, 0)

# Calculate and print the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy * 100)
print("Weights:", weights)


In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import itertools
import random

def single_perceptron(TEST):
    # Define the number of bits
    no_bit = 4
    max_epoch = 100
    # Generate all possible binary strings of length no_bit
    ds = [''.join(p) for p in itertools.product('01', repeat=no_bit)][1:]
    # Generate random labels for each binary string
    labels = [random.randint(0, 1) for i in range(len(ds))]

    # Create the dataset
    data = [[int(char) for char in string] + [label] for string, label in zip(ds, labels)]
    #print(data)

    # Convert the dataset to a DataFrame
    columns = [f"X{i+1}" for i in range(no_bit)] + ["Output"]
    data_df = pd.DataFrame(data, columns=columns)

    # Split the dataset into training and testing
    X = data_df.iloc[:, :-1].values
    y = data_df.iloc[:, -1].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST/100, random_state=42)

    # Make a prediction with weights
    def predict(row, weights, th):
        activation = -1*th
        for i in range(len(row)-1):
            activation += weights[i] * row[i]
        return 1.0 if activation >= 0.0 else 0.0

    # Define the perceptron function
    def adapt_weights(arr, w, th, count):
        count += 1
        flag = 0
        for i in range(arr.shape[0]):
            total_sum = 0.0
            for j in range(arr.shape[1]-1):
                total_sum += arr[i][j] * w[j]

            predicted_output = 0
            desired_output = arr[i][arr.shape[1]-1]

            if total_sum < th:
                predicted_output = 0
            else:
                predicted_output = 1

            if predicted_output == 0 and desired_output == 1:
                flag = 1
                for j in range(arr.shape[1]-1):
                    w[j] += arr[i][j]
                break
            elif predicted_output == 1 and desired_output == 0:
                flag = 1
                for j in range(arr.shape[1]-1):
                    w[j] -= arr[i][j]
                break

        if flag == 1 and count <= max_epoch:
            w = adapt_weights(arr, w, th, count)

        return w

    # Train the perceptron model
    th = 0.5
    weights = adapt_weights(X_train, np.random.rand(len(X_train[0])), th, 0)
    print("Weights:", weights)

    cnt=0
    for row in X_test:
        actual_output = predict(row,weights,th)
        desired_output = row[-1]
        #print("de",desired_output,"ac",actual_output)
        if(actual_output==desired_output):
            cnt+=1

    print("Accuracy: ",cnt/len(X_test)*100.0)


single_perceptron(20)
single_perceptron(25)
single_perceptron(40)

[[0, 0, 0, 1, 1], [0, 0, 1, 0, 1], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 0], [1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 1], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]
Weights: [ 0.95878558 -1.81579495 -0.16624933  0.4376756 ]
de 0 ac 1.0
de 0 ac 0.0
de 1 ac 0.0
Accuracy:  33.33333333333333
[[0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 1], [0, 1, 0, 1, 1], [0, 1, 1, 0, 0], [0, 1, 1, 1, 1], [1, 0, 0, 0, 1], [1, 0, 0, 1, 0], [1, 0, 1, 0, 0], [1, 0, 1, 1, 0], [1, 1, 0, 0, 0], [1, 1, 0, 1, 1], [1, 1, 1, 0, 1], [1, 1, 1, 1, 1]]
Weights: [ 0.94414464 -0.99017174  1.39288487  0.53434426]
de 0 ac 1.0
de 0 ac 0.0
de 1 ac 0.0
de 0 ac 1.0
Accuracy:  25.0
[[0, 0, 0, 1, 0], [0, 0, 1, 0, 0], [0, 0, 1, 1, 0], [0, 1, 0, 0, 0], [0, 1, 0, 1, 0], [0, 1, 1, 0, 0], [0, 1, 1, 1, 0], [1, 0, 0, 0, 1], [1, 0, 0, 1, 1], [1, 0, 1, 0, 0], [1, 0, 1, 1, 1], [1, 1, 0, 0, 1], [1, 1, 0, 1, 0], [1, 1, 1, 0, 0], [1